In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy as np
import os
from time import time
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import matplotlib.animation as animation

In [2]:
#preprocessing vid to npy
def getOpticalFlow(video):
    """Calculate dense optical flow of input video
    Args:
        video: the input video with shape of [frames,height,width,channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames,height,width,channel]
        flows_y: the optical flow at y-axis, with the shape of [frames,height,width,channel]
    """
    # initialize the list of optical flows
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img,(224,224,1)))

    flows = []
    for i in range(0,len(video)-1):
        # calculate optical flow between each pair of frames
        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i+1], None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0],None,0,255,cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1],None,0,255,cv2.NORM_MINMAX)
        # Add into list 
        flows.append(flow)
        
    # Padding the last frame as empty array
    flows.append(np.zeros((224,224,2)))
      
    return np.array(flows, dtype=np.float32)

In [3]:
def Video2Npy(file_path, resize=(224,224)):
    """Load video and tansfer it into .npy format
    Args:
        file_path: the path of video file
        resize: the target resolution of output video
    Returns:
        frames: gray-scale video
        flows: magnitude video of optical flows 
    """
    # Load video
    cap = cv2.VideoCapture(file_path)
    # Get number of frames
    len_frames = int(cap.get(7))
    # Extract frames from video
    fin = []
    five = 140
    i = 0
    try:
        while i < (len_frames-1) and (i+140) <(len_frames-1):
            frames = []
            while i < (five-1):
                _, frame = cap.read()
                frame = cv2.resize(frame,resize, interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = np.reshape(frame, (224,224,3))
                frames.append(frame)
                i+=1   
            frames = np.array(frames)
            five = five + 140
            flows = getOpticalFlow(frames)
    
            result = np.zeros((len(flows),224,224,5))
            result[...,:3] = frames
            result[...,3:] = flows
            fin.append(result)
    except:
        print("Error: ", file_path, len_frames,i)
    finally:
        
        cap.release() 
    return fin

In [4]:

def Save2Npy(file_dir, save_dir):
    """Transfer all the videos and save them into specified directory
    Args:
        file_dir: source folder of target videos
        save_dir: destination folder of output .npy files
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # List the files
    videos = os.listdir(file_dir)
    for v in tqdm(videos):
        # Split video name
        video_name = v.split('.')[0]
        # Get src 
        video_path = os.path.join(file_dir, v)
        # Get dest 
         
        # Load and preprocess video
        data = Video2Npy(file_path=video_path, resize=(224,224))
        i = 0
        while i < len(data)-1:
            data[i] = np.uint8(data[i])
            # Save as .npy file
            save_path = os.path.join(save_dir, str(i)+'.npy')
            np.save(save_path, data[i])
            i +=1
    
    return None

In [5]:
source_path = 'E:\\dataset rwf-2000\\RWF2000-Video-Database-for-Violence-Detection-master\\RWF2000-Video-Database-for-Violence-Detection-master\\random testing\\data\\2'
target_path = 'E:\\dataset rwf-2000\\RWF2000-Video-Database-for-Violence-Detection-master\\RWF2000-Video-Database-for-Violence-Detection-master\\random testing\\npy\\2'
Save2Npy(file_dir=source_path, save_dir=target_path)

100%|██████████| 1/1 [05:20<00:00, 320.14s/it]


In [5]:
def normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data-mean) / std

def random_flip(video, prob):
    s = np.random.rand()
    if s < prob:
        video = np.flip(m=video, axis=2)
    return video    

def uniform_sampling(video, target_frames=64):
    # get total frames of input video and calculate sampling interval 
    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    padding = []
    if num_pad>0:
        for i in range(-num_pad,0):
            try: 
                padding.append(video[i])
            except:
                padding.append(video[0])
        sampled_video += padding     
    # get sampled video
    return np.array(sampled_video, dtype=np.float32)

def random_clip( video, target_frames=64):
    start_point = np.random.randint(len(video)-target_frames)
    return video[start_point:start_point+target_frames]

def dynamic_crop(video):
    # extract layer of optical flow from video
    opt_flows = video[...,3]
    # sum of optical flow magnitude of individual frame
    magnitude = np.sum(opt_flows, axis=0)
    # filter slight noise by threshold 
    thresh = np.mean(magnitude)
    magnitude[magnitude<thresh] = 0
    # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
    x_pdf = np.sum(magnitude, axis=1) + 0.001
    y_pdf = np.sum(magnitude, axis=0) + 0.001
    # normalize PDF of x and y so that the sum of probs = 1
    x_pdf /= np.sum(x_pdf)
    y_pdf /= np.sum(y_pdf)
    # randomly choose some candidates for x and y 
    x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
    y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
    # get the mean of x and y coordinates for better robustness
    x = int(np.mean(x_points))
    y = int(np.mean(y_points))
    # avoid to beyond boundaries of array
    x = max(56,min(x,167))
    y = max(56,min(y,167))
    # get cropped video 
    return video[:,x-56:x+56,y-56:y+56,:]  

def color_jitter(video):
    # range of s-component: 0-1
    # range of v component: 0-255
    s_jitter = np.random.uniform(-0.2,0.2)
    v_jitter = np.random.uniform(-30,30)
    for i in range(len(video)):
        hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
        s = hsv[...,1] + s_jitter
        v = hsv[...,2] + v_jitter
        s[s<0] = 0
        s[s>1] = 1
        v[v<0] = 0
        v[v>255] = 255
        hsv[...,1] = s
        hsv[...,2] = v
        video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return video
    
def load_data(path):
    # load the processed .npy files which have 5 channels (1-3 for RGB, 4-5 for optical flows)
    data = np.load(path, mmap_mode='r')
    data = np.float32(data)
    # sampling 64 frames uniformly from the entire video
    data = uniform_sampling(video=data, target_frames=64)
    # whether to utilize the data augmentation
    data[...,:3] = color_jitter(data[...,:3])
    data = random_flip(data, prob=0.5)
    # normalize rgb images and optical flows, respectively
    data[...,:3] = normalize(data[...,:3])
    data[...,3:] = normalize(data[...,3:])
    return data

# extract the rgb images 
def get_rgb(input_x):
    rgb = input_x[...,:3]
    return rgb

# extract the optical flows
def get_opt(input_x):
    opt= input_x[...,3:5]
    return opt


In [8]:
from keras.models import load_model
from tensorflow.keras.optimizers import SGD

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = load_model('E:\work\grad new\grad\code\Models\keras_model_new.h5', compile = False)


ValueError: bad marshal data (unknown type code)

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 224, 224 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 64, 224, 224, 0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 64, 224, 224, 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 224, 224, 448         lambda_1[0][0]                   
____________________________________________________________________________________________

In [13]:
target_path = 'E:\\dataset rwf-2000\\RWF2000-Video-Database-for-Violence-Detection-master\\RWF2000-Video-Database-for-Violence-Detection-master\\random testing\\npy\\2'
output = []
videos = os.listdir(target_path)
for v in tqdm(videos):
        testPath = os.path.join(target_path,v)
        Xinput = load_data(testPath)
        prediction = model.predict(np.reshape(Xinput,[1,64,224,224,5]))
        output.append(prediction)

100%|██████████| 21/21 [02:14<00:00,  6.43s/it]


In [14]:
import pandas as pd
prediction = np.array(output)
prediction = np.squeeze(prediction)
df = pd.DataFrame(data=prediction,columns=['Fight_Prob','NonFight_Prob'])
df

,Fight_Prob,NonFight_Prob
0,0.074160,0.925840
1,0.697042,0.302958
2,0.495993,0.504007
3,0.242086,0.757914
4,0.654369,0.345631
5,0.701343,0.298657
6,0.189484,0.810516
7,0.348688,0.651312
8,0.513260,0.486740
9,0.098758,0.901242


In [15]:
df.to_csv('E:\\dataset rwf-2000\\RWF2000-Video-Database-for-Violence-Detection-master\\RWF2000-Video-Database-for-Violence-Detection-master\\random testing\\data\\out.csv') 